In [ ]:
import psycopg2
import xml.etree.ElementTree as ElementTree
import sys

In [ ]:
repertoire_racine = '/home/michel/legi/plat/'
version_courante = '1'

In [ ]:
def parse_struct(contenu):
    valeurs = {}
    
    TEXTELR = ElementTree.fromstring(contenu)
    assert TEXTELR.tag == 'TEXTELR'

    #
    META = TEXTELR[0]
    assert META.tag == 'META'

    ##
    META_COMMUN = META[0]
    assert META_COMMUN.tag == 'META_COMMUN'

    ###
    ID = META_COMMUN[0]
    assert ID.tag == 'ID'
    valeurs['ID'] = ID.text

    id_eli_present = 0
    valeurs['ID_ELI'] = ''
    if META_COMMUN[1].tag == 'ID_ELI':
        id_eli_present = 1
        ID_ELI = META_COMMUN[1]
        valeurs['ID_ELI'] = ID_ELI.text
        
    eli_alias_present = 0
    valeurs['ID_ELI_ALIAS'] = ''
    if META_COMMUN[1 + id_eli_present].tag == 'ELI_ALIAS':
        eli_alias_present = 1
        ELI_ALIAS = META_COMMUN[1 + id_eli_present]
        assert len(list(ELI_ALIAS)) == 1
        ID_ELI_ALIAS = ELI_ALIAS[0]
        assert ID_ELI_ALIAS.tag == 'ID_ELI_ALIAS'
        valeurs['ID_ELI_ALIAS'] = ID_ELI_ALIAS.text
        
    ANCIEN_ID = META_COMMUN[1 + id_eli_present + eli_alias_present]
    assert ANCIEN_ID.tag == 'ANCIEN_ID'
    valeurs['ANCIEN_ID'] = ANCIEN_ID.text

    ORIGINE = META_COMMUN[2 + id_eli_present + eli_alias_present]
    assert ORIGINE.tag == 'ORIGINE'
    assert ORIGINE.text == 'LEGI'

    URL = META_COMMUN[3 + id_eli_present + eli_alias_present]
    assert URL.tag == 'URL'
    valeurs['URL'] = URL.text

    NATURE = META_COMMUN[4 + id_eli_present + eli_alias_present]
    assert NATURE.tag == 'NATURE'
    valeurs['NATURE'] = NATURE.text

    ##
    META_SPEC = META[1]
    assert META_SPEC.tag == 'META_SPEC'

    ###
    META_TEXTE_CHRONICLE = META_SPEC[0]
    assert META_TEXTE_CHRONICLE.tag == 'META_TEXTE_CHRONICLE'

    ####
    CID = META_TEXTE_CHRONICLE[0]
    assert CID.tag == 'CID'
    valeurs['CID'] = CID.text

    NUM = META_TEXTE_CHRONICLE[1]
    assert NUM.tag == 'NUM'
    valeurs['NUM'] = NUM.text

    NUM_SEQUENCE = META_TEXTE_CHRONICLE[2]
    assert NUM_SEQUENCE.tag == 'NUM_SEQUENCE'
    valeurs['NUM_SEQUENCE'] = NUM_SEQUENCE.text

    NOR = META_TEXTE_CHRONICLE[3]
    assert NOR.tag == 'NOR'
    valeurs['NOR'] = NOR.text

    DATE_PUBLI = META_TEXTE_CHRONICLE[4]
    assert DATE_PUBLI.tag == 'DATE_PUBLI'
    valeurs['DATE_PUBLI'] = DATE_PUBLI.text

    DATE_TEXTE = META_TEXTE_CHRONICLE[5]
    assert DATE_TEXTE.tag == 'DATE_TEXTE'
    valeurs['DATE_TEXTE'] = DATE_TEXTE.text

    der_modif_present = 0
    valeurs['DERNIERE_MODIFICATION'] = ''
    if META_TEXTE_CHRONICLE[6].tag == 'DERNIERE_MODIFICATION':
        der_modif_present = 1
        DERNIERE_MODIFICATION = META_TEXTE_CHRONICLE[6]
        valeurs['DERNIERE_MODIFICATION'] = DERNIERE_MODIFICATION.text
    
    versions_a_venir_present = 0
    valeurs['VERSIONS_A_VENIR'] = []
    if META_TEXTE_CHRONICLE[6 + der_modif_present].tag == 'VERSIONS_A_VENIR':
        versions_a_venir_present = 1
        VERSIONS_A_VENIR = META_TEXTE_CHRONICLE[6 + der_modif_present]
        
        #####
        for VERSION_A_VENIR in VERSIONS_A_VENIR:
            assert VERSION_A_VENIR.tag == 'VERSION_A_VENIR'
            valeurs['VERSIONS_A_VENIR'].append(VERSION_A_VENIR.text)
    
    ORIGINE_PUBLI = META_TEXTE_CHRONICLE[6 + der_modif_present + versions_a_venir_present]
    assert ORIGINE_PUBLI.tag == 'ORIGINE_PUBLI'
    valeurs['ORIGINE_PUBLI'] = ORIGINE_PUBLI.text
    
    PAGE_DEB_PUBLI = META_TEXTE_CHRONICLE[7 + der_modif_present + versions_a_venir_present]
    assert PAGE_DEB_PUBLI.tag == 'PAGE_DEB_PUBLI'
    valeurs['PAGE_DEB_PUBLI'] = PAGE_DEB_PUBLI.text
    
    PAGE_FIN_PUBLI = META_TEXTE_CHRONICLE[8 + der_modif_present + versions_a_venir_present]
    assert PAGE_FIN_PUBLI.tag == 'PAGE_FIN_PUBLI'
    valeurs['PAGE_FIN_PUBLI'] = PAGE_FIN_PUBLI.text

    #
    VERSIONS = TEXTELR[1]
    assert VERSIONS.tag == 'VERSIONS'
    
    ##
    valeurs['VERSIONS'] = []
    for VERSION in VERSIONS:
        assert VERSION.tag == 'VERSION'
        
        VERSION_data = {}
        
        VERSION_data['etat'] = VERSION.attrib['cid']
        
        ###
        LIEN_TXT = VERSION[0]
        assert LIEN_TXT.tag == 'LIEN_TXT'
        
        VERSION_data['debut'] = LIEN_TXT.attrib['debut']
        VERSION_data['fin'] = LIEN_TXT.attrib['fin']
        VERSION_data['id_'] = LIEN_TXT.attrib['id']
        VERSION_data['num'] = LIEN_TXT.attrib['num']
        
        valeurs['VERSIONS'].append(VERSION_data)
        
    #
    STRUCT = TEXTELR[2]
    assert STRUCT.tag == 'STRUCT'
    
    ##
    valeurs['LIENS_ART'] = []
    valeurs['LIENS_SECTION_TA'] = []
    for LIEN in STRUCT:
        if LIEN.tag == 'LIEN_ART':
            LIEN_ART_data = {}

            LIEN_ART_data['debut'] = LIEN.attrib['debut']
            LIEN_ART_data['etat'] = LIEN.attrib['etat']
            LIEN_ART_data['fin'] = LIEN.attrib['fin']
            LIEN_ART_data['id_'] = LIEN.attrib['id']
            LIEN_ART_data['num'] = LIEN.attrib['num']
            LIEN_ART_data['origine'] = LIEN.attrib['origine']

            valeurs['LIENS_ART'].append(LIEN_ART_data)
            
        elif LIEN.tag == 'LIEN_SECTION_TA':
            LIEN_SECTION_TA_data = {}

            LIEN_SECTION_TA_data['cid'] = LIEN.attrib['cid']
            LIEN_SECTION_TA_data['debut'] = LIEN.attrib['debut']
            LIEN_SECTION_TA_data['etat'] = LIEN.attrib['etat']
            LIEN_SECTION_TA_data['fin'] = LIEN.attrib['fin']
            LIEN_SECTION_TA_data['id_'] = LIEN.attrib['id']
            LIEN_SECTION_TA_data['niv'] = LIEN.attrib['niv']
            LIEN_SECTION_TA_data['url'] = LIEN.attrib['url']

            valeurs['LIENS_SECTION_TA'].append(LIEN_SECTION_TA_data)
            
        else:
            raise ValueError(LIEN.TAG)

    return valeurs

In [ ]:
def parse_version(contenu):
    valeurs = {}
    
    TEXTE_VERSION = ElementTree.fromstring(contenu)
    assert TEXTE_VERSION.tag == 'TEXTE_VERSION'

    #
    META = TEXTE_VERSION[0]
    assert META.tag == 'META'

    ##
    META_COMMUN = META[0]
    assert META_COMMUN.tag == 'META_COMMUN'

    ###
    ID = META_COMMUN[0]
    assert ID.tag == 'ID'
    valeurs['ID'] = ID.text

    id_eli_present = 0
    valeurs['ID_ELI'] = ''
    if META_COMMUN[1].tag == 'ID_ELI':
        id_eli_present = 1
        ID_ELI = META_COMMUN[1]
        valeurs['ID_ELI'] = ID_ELI.text
        
    eli_alias_present = 0
    valeurs['ID_ELI_ALIAS'] = ''
    if META_COMMUN[1 + id_eli_present].tag == 'ELI_ALIAS':
        eli_alias_present = 1
        ELI_ALIAS = META_COMMUN[1 + id_eli_present]
        assert len(list(ELI_ALIAS)) == 1
        ID_ELI_ALIAS = ELI_ALIAS[0]
        assert ID_ELI_ALIAS.tag == 'ID_ELI_ALIAS'
        valeurs['ID_ELI_ALIAS'] = ID_ELI_ALIAS.text
        
    ANCIEN_ID = META_COMMUN[1 + id_eli_present + eli_alias_present]
    assert ANCIEN_ID.tag == 'ANCIEN_ID'
    valeurs['ANCIEN_ID'] = ANCIEN_ID.text

    ORIGINE = META_COMMUN[2 + id_eli_present + eli_alias_present]
    assert ORIGINE.tag == 'ORIGINE'
    assert ORIGINE.text == 'LEGI'

    URL = META_COMMUN[3 + id_eli_present + eli_alias_present]
    assert URL.tag == 'URL'
    valeurs['URL'] = URL.text

    NATURE = META_COMMUN[4 + id_eli_present + eli_alias_present]
    assert NATURE.tag == 'NATURE'
    valeurs['NATURE'] = NATURE.text

    ##
    META_SPEC = META[1]
    assert META_SPEC.tag == 'META_SPEC'

    ###
    META_TEXTE_CHRONICLE = META_SPEC[0]
    assert META_TEXTE_CHRONICLE.tag == 'META_TEXTE_CHRONICLE'

    ####
    CID = META_TEXTE_CHRONICLE[0]
    assert CID.tag == 'CID'
    valeurs['CID'] = CID.text

    NUM = META_TEXTE_CHRONICLE[1]
    assert NUM.tag == 'NUM'
    valeurs['NUM'] = NUM.text

    NUM_SEQUENCE = META_TEXTE_CHRONICLE[2]
    assert NUM_SEQUENCE.tag == 'NUM_SEQUENCE'
    valeurs['NUM_SEQUENCE'] = NUM_SEQUENCE.text

    NOR = META_TEXTE_CHRONICLE[3]
    assert NOR.tag == 'NOR'
    valeurs['NOR'] = NOR.text

    DATE_PUBLI = META_TEXTE_CHRONICLE[4]
    assert DATE_PUBLI.tag == 'DATE_PUBLI'
    valeurs['DATE_PUBLI'] = DATE_PUBLI.text

    DATE_TEXTE = META_TEXTE_CHRONICLE[5]
    assert DATE_TEXTE.tag == 'DATE_TEXTE'
    valeurs['DATE_TEXTE'] = DATE_TEXTE.text

    der_modif_present = 0
    valeurs['DERNIERE_MODIFICATION'] = ''
    if META_TEXTE_CHRONICLE[6].tag == 'DERNIERE_MODIFICATION':
        der_modif_present = 1
        DERNIERE_MODIFICATION = META_TEXTE_CHRONICLE[6]
        valeurs['DERNIERE_MODIFICATION'] = DERNIERE_MODIFICATION.text
    
    versions_a_venir_present = 0
    valeurs['VERSIONS_A_VENIR'] = []
    if META_TEXTE_CHRONICLE[6 + der_modif_present].tag == 'VERSIONS_A_VENIR':
        versions_a_venir_present = 1
        VERSIONS_A_VENIR = META_TEXTE_CHRONICLE[6 + der_modif_present]
        
        #####
        for VERSION_A_VENIR in VERSIONS_A_VENIR:
            assert VERSION_A_VENIR.tag == 'VERSION_A_VENIR'
            valeurs['VERSIONS_A_VENIR'].append(VERSION_A_VENIR.text)
    
    ORIGINE_PUBLI = META_TEXTE_CHRONICLE[6 + der_modif_present + versions_a_venir_present]
    assert ORIGINE_PUBLI.tag == 'ORIGINE_PUBLI'
    valeurs['ORIGINE_PUBLI'] = ORIGINE_PUBLI.text
    
    PAGE_DEB_PUBLI = META_TEXTE_CHRONICLE[7 + der_modif_present + versions_a_venir_present]
    assert PAGE_DEB_PUBLI.tag == 'PAGE_DEB_PUBLI'
    valeurs['PAGE_DEB_PUBLI'] = PAGE_DEB_PUBLI.text
    
    PAGE_FIN_PUBLI = META_TEXTE_CHRONICLE[8 + der_modif_present + versions_a_venir_present]
    assert PAGE_FIN_PUBLI.tag == 'PAGE_FIN_PUBLI'
    valeurs['PAGE_FIN_PUBLI'] = PAGE_FIN_PUBLI.text

    ###
    META_TEXTE_VERSION = META_SPEC[1]
    assert META_TEXTE_VERSION.tag == 'META_TEXTE_VERSION'

    ####
    TITRE = META_TEXTE_VERSION[0]
    assert TITRE.tag == 'TITRE'
    valeurs['TITRE'] = TITRE.text

    TITREFULL = META_TEXTE_VERSION[1]
    assert TITREFULL.tag == 'TITREFULL'
    valeurs['TITREFULL'] = TITREFULL.text

    ETAT = META_TEXTE_VERSION[2]
    assert ETAT.tag == 'ETAT'
    valeurs['ETAT'] = ETAT.text

    DATE_DEBUT = META_TEXTE_VERSION[3]
    assert DATE_DEBUT.tag == 'DATE_DEBUT'
    valeurs['DATE_DEBUT'] = DATE_DEBUT.text

    DATE_FIN = META_TEXTE_VERSION[4]
    assert DATE_FIN.tag == 'DATE_FIN'
    valeurs['DATE_FIN'] = DATE_FIN.text

    AUTORITE = META_TEXTE_VERSION[5]
    assert AUTORITE.tag == 'AUTORITE'
    valeurs['AUTORITE'] = AUTORITE.text

    MINISTERE = META_TEXTE_VERSION[6]
    assert MINISTERE.tag == 'MINISTERE'
    valeurs['MINISTERE'] = MINISTERE.text

    liens_present = 0
    valeurs['LIENS'] = ''
    assert len(META_TEXTE_VERSION) in [7, 8]
    if len(META_TEXTE_VERSION) == 8:
        liens_present = 1
        LIENS = META_TEXTE_VERSION[7]
        assert LIENS.tag == 'LIENS'
        
        #####
        for LIEN in LIENS:
            assert LIEN.tag == 'LIEN'

            LIEN_data = {}

            LIEN_data['cidtexte'] = LIEN.attrib['cidtexte']
            LIEN_data['datesignatexte'] = LIEN.attrib['datesignatexte']
            LIEN_data['id_'] = LIEN.attrib['id']
            LIEN_data['naturetexte'] = LIEN.attrib['naturetexte']
            LIEN_data['nortexte'] = LIEN.attrib['nortexte']
            LIEN_data['num'] = LIEN.attrib['num']
            LIEN_data['numtexte'] = LIEN.attrib['numtexte']
            LIEN_data['sens'] = LIEN.attrib['sens']
            LIEN_data['typelien'] = LIEN.attrib['typelien']
            LIEN_data['texte'] = LIEN.text

            valeurs['LIENS'].append(LIEN_data)
      
    #
    VISAS = TEXTE_VERSION[1]
    assert VISAS.tag == 'VISAS'

    ##
    CONTENU = VISAS[0]
    valeurs['VISAS'] = ElementTree.tostring(CONTENU, encoding='unicode', method='xml')
      
    #
    SIGNATAIRES = TEXTE_VERSION[2]
    assert SIGNATAIRES.tag == 'SIGNATAIRES'

    ##
    CONTENU = SIGNATAIRES[0]
    valeurs['SIGNATAIRES'] = ElementTree.tostring(CONTENU, encoding='unicode', method='xml')
      
    #
    TP = TEXTE_VERSION[3]
    assert TP.tag == 'TP'

    ##
    CONTENU = TP[0]
    valeurs['TP'] = ElementTree.tostring(CONTENU, encoding='unicode', method='xml')
      
    #
    NOTA = TEXTE_VERSION[4]
    assert NOTA.tag == 'NOTA'

    ##
    CONTENU = NOTA[0]
    valeurs['NOTA'] = ElementTree.tostring(CONTENU, encoding='unicode', method='xml')
      
    #
    ABRO = TEXTE_VERSION[5]
    assert ABRO.tag == 'ABRO'

    ##
    CONTENU = ABRO[0]
    valeurs['ABRO'] = ElementTree.tostring(CONTENU, encoding='unicode', method='xml')
      
    #
    RECT = TEXTE_VERSION[6]
    assert RECT.tag == 'RECT'

    ##
    CONTENU = RECT[0]
    valeurs['RECT'] = ElementTree.tostring(CONTENU, encoding='unicode', method='xml')
            
    return valeurs

In [3]:

connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

while True:
    curseur.execute("select * from article where (status is null or status != %s) and (mod(numero_ligne, %s) = %s) limit 1;", (version_courante, nb_proc, id_proc))
    resultat_requete = curseur.fetchone()
    
    if not resultat_requete:
        break
        
    nom_fichier = resultat_requete[0]
        
    with open(repertoire_racine + 'article/' + nom_fichier) as f:
        contenu = f.read()
    
    valeurs = parse_contenu(contenu)
    
    valeurs['status'] = version_courante
    valeurs['nom_fichier'] = nom_fichier

    curseur.execute("""
        update article set
            ID = %(ID)s,
            ID_ELI = %(ID_ELI)s,
            ID_ELI_ALIAS = %(ID_ELI_ALIAS)s,
            ANCIEN_ID = %(ANCIEN_ID)s,
            MCS_ART = %(MCS_ART)s,
            NUM = %(NUM)s,
            TYPE = %(TYPE)s,
            URL = %(URL)s,
            TEXTE_cid = %(TEXTE_cid)s,
            BLOC_TEXTUEL = %(BLOC_TEXTUEL)s,
            DATE_DEBUT = %(DATE_DEBUT)s,
            DATE_FIN = %(DATE_FIN)s,
            status = %(status)s
        where nom_fichier = %(nom_fichier)s;
        """, valeurs)
    connection.commit()


curseur.close()
connection.close()

In [ ]:
nom_fichier

In [ ]:
import xml.dom.minidom
xml_parsed = xml.dom.minidom.parseString(contenu)
print(xml_parsed.toprettyxml())

In [8]:
connection.rollback()

In [ ]:
nom_fichier = 'JORFARTI000000961438.xml'

with open(repertoire_racine + 'article/' + nom_fichier) as f:
    contenu = f.read()
    
ARTICLE = ElementTree.fromstring(contenu)
assert ARTICLE.tag == 'ARTICLE'
#
BLOC_TEXTUEL = ARTICLE[4]
assert BLOC_TEXTUEL.tag == 'BLOC_TEXTUEL'

##
CONTENU = BLOC_TEXTUEL[0]


In [ ]:
print()

In [ ]:
CONTENU.items()

In [ ]:
list(CONTENU)

In [ ]:
curseur.close()
connection.close()